<a href="https://colab.research.google.com/github/l-guilherme/PLN/blob/main/PLN_T2_ner_crf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!python -m pip install --upgrade pip
!pip install seqeval
!pip install -U sklearn_crfsuite
!pip install scikit-learn==1.3.2

!python -m spacy download pt_core_news_sm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 22.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=7bbf18c9d864752db3a8cfbec01d78ff9aa7d97dd451f2991192c7659d64a3c3
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 116.7 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 72.1 MB/s eta 0:00:00
✔ Download and installation successful


In [2]:
import os
import numpy as np

from sklearn_crfsuite import CRF
from seqeval.metrics import classification_report

In [3]:
!wget https://raw.githubusercontent.com/messias077/ner_pt/main/data/corpora/le_ner/train.conll -P le_ner/
!wget https://raw.githubusercontent.com/messias077/ner_pt/main/data/corpora/le_ner/test.conll -P le_ner/

!wget https://raw.githubusercontent.com/ulysses-camara/ulysses-ner-br/refs/heads/main/PL-corpus_v1/pl_corpus_categorias/train.txt -P ulysses_ner/
!wget https://raw.githubusercontent.com/ulysses-camara/ulysses-ner-br/refs/heads/main/PL-corpus_v1/pl_corpus_categorias/test.txt -P ulysses_ner/

--2025-02-03 10:32:57--  https://raw.githubusercontent.com/messias077/ner_pt/main/data/corpora/le_ner/train.conll
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2142199 (2.0M) [text/plain]
Saving to: ‘le_ner/train.conll’

train.conll         100%[===================>]   2.04M  --.-KB/s    in 0.03s   

2025-02-03 10:32:57 (77.5 MB/s) - ‘le_ner/train.conll’ saved [2142199/2142199]

--2025-02-03 10:32:58--  https://raw.githubusercontent.com/messias077/ner_pt/main/data/corpora/le_ner/test.conll
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK

## Função para leitura da base de dados no padrão BIO

In [4]:
def read_corpus_file(corpus_file, delimiter='\t', ner_column=1):
    with open(corpus_file, encoding='utf-8') as file:
        lines = file.readlines()
    data = []
    words = []
    tags = []
    for line in lines:
        line = line.replace('\n', '')
        if line != '':
            if delimiter in line:
                fragments = line.split(delimiter)
                words.append(fragments[0])
                tags.append(fragments[ner_column])
        else:
            if len(words) > 1:
                data.append((words, tags))
            words = []
            tags = []
    return data

In [5]:
corpus_name = 'le_ner'
#corpus_name = 'ulysses_ner'

report_dir = 'report/'

train_file = None
test_file = None

id_ner = 1
delimiter = ' '

if corpus_name == 'le_ner':
  train_file = '/content/le_ner/train.conll'
  test_file = '/content/le_ner/test.conll'
elif corpus_name == 'ulysses_ner':
  train_file = '/content/ulysses_ner/train.txt'
  test_file = '/content/ulysses_ner/test.txt'

print(f'\nCorpus: {corpus_name}')

report_dir = os.path.join(report_dir, corpus_name)

os.makedirs(report_dir, exist_ok=True)

train_data = read_corpus_file(train_file, delimiter=delimiter, ner_column=id_ner)
test_data = read_corpus_file(test_file, delimiter=delimiter, ner_column=id_ner)

print(f'\nTrain data: {len(train_data)}')
print(f'Test data: {len(test_data)}')

test_data_original = np.array(test_data, dtype=object)


Corpus: le_ner

Train data: 7821
Test data: 1389


In [6]:
train_data[0]

(['EMENTA',
  ':',
  'APELAÇÃO',
  'CÍVEL',
  '-',
  'AÇÃO',
  'DE',
  'INDENIZAÇÃO',
  'POR',
  'DANOS',
  'MORAIS',
  '-',
  'PRELIMINAR',
  '-',
  'ARGUIDA',
  'PELO',
  'MINISTÉRIO',
  'PÚBLICO',
  'EM',
  'GRAU',
  'RECURSAL',
  '-',
  'NULIDADE',
  '-',
  'AUSÊNCIA',
  'DE',
  'INTERVENÇÃO',
  'DO',
  'PARQUET',
  'NA',
  'INSTÂNCIA',
  'A',
  'QUO',
  '-',
  'PRESENÇA',
  'DE',
  'INCAPAZ',
  '-',
  'PREJUÍZO',
  'EXISTENTE',
  '-',
  'PRELIMINAR',
  'ACOLHIDA',
  '-',
  'NULIDADE',
  'RECONHECIDA',
  '.'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-ORGANIZACAO',
  'I-ORGANIZACAO',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'])

## Função que executa o pré-processamento do corpus usando a ferramenta Spacy

In [7]:
import spacy

def data_preprocessing(data):
    nlp = spacy.load(name='pt_core_news_sm',
                     disable=['parser', 'ner', 'lemmatizer', 'textcat'])
    preprocessed_data = []
    for d in data:
        sentence = ' '.join(d[0])
        doc = nlp(sentence)
        pos_tags = [t.pos_ for t in doc]
        preprocessed_data.append((d[0], pos_tags, d[1]))
    return preprocessed_data

In [8]:
train_data = data_preprocessing(train_data)

test_data = data_preprocessing(test_data)

In [9]:
train_data[0]

(['EMENTA',
  ':',
  'APELAÇÃO',
  'CÍVEL',
  '-',
  'AÇÃO',
  'DE',
  'INDENIZAÇÃO',
  'POR',
  'DANOS',
  'MORAIS',
  '-',
  'PRELIMINAR',
  '-',
  'ARGUIDA',
  'PELO',
  'MINISTÉRIO',
  'PÚBLICO',
  'EM',
  'GRAU',
  'RECURSAL',
  '-',
  'NULIDADE',
  '-',
  'AUSÊNCIA',
  'DE',
  'INTERVENÇÃO',
  'DO',
  'PARQUET',
  'NA',
  'INSTÂNCIA',
  'A',
  'QUO',
  '-',
  'PRESENÇA',
  'DE',
  'INCAPAZ',
  '-',
  'PREJUÍZO',
  'EXISTENTE',
  '-',
  'PRELIMINAR',
  'ACOLHIDA',
  '-',
  'NULIDADE',
  'RECONHECIDA',
  '.'],
 ['PROPN',
  'PUNCT',
  'PROPN',
  'PROPN',
  'PUNCT',
  'PROPN',
  'ADP',
  'PROPN',
  'ADP',
  'PROPN',
  'PROPN',
  'PUNCT',
  'PROPN',
  'PUNCT',
  'VERB',
  'ADP',
  'PROPN',
  'PROPN',
  'ADP',
  'PROPN',
  'PROPN',
  'PUNCT',
  'PROPN',
  'PUNCT',
  'X',
  'ADP',
  'PROPN',
  'ADP',
  'PROPN',
  'ADP',
  'NOUN',
  'DET',
  'PROPN',
  'PUNCT',
  'PROPN',
  'ADP',
  'PROPN',
  'PUNCT',
  'PROPN',
  'PROPN',
  'PUNCT',
  'PROPN',
  'PROPN',
  'PUNCT',
  'PROPN',
  'NOUN',

## Funções usadas para extrair as features dos tokens e de seus vizinhos.

In [10]:
def extract_sent_features(sentence):
    return [extract_features(sentence, i) for i in range(len(sentence))]


def extract_labels(sentence):
    return [label for _, _, label in sentence]


def extract_features(sentence, i):
    word = sentence[i][0]
    postag = sentence[i][1]
    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
        'word.islower()': word.islower(),
        'word[0].isupper()': word[0].isupper(),
        'word[0].islower()': word[0].islower(),
        'not word[0].isalnum()': not word[0].isalnum(),
        'not word.isalnum()': not word.isalnum(),
        'word.isalpha()': word.isalpha()
    }
    if i > 0:
        word1 = sentence[i - 1][0]
        postag1 = sentence[i - 1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
            '-1:word.islower()': word1.islower()
        })
    else:
        features['BOS'] = True # BOS = Begin of Sentence
    if i > 1:
        word1 = sentence[i - 2][0]
        postag1 = sentence[i - 2][1]
        features.update({
            '-2:word.lower()': word1.lower(),
            '-2:word.istitle()': word1.istitle(),
            '-2:word.isupper()': word1.isupper(),
            '-2:postag': postag1,
            '-2:postag[:2]': postag1[:2],
            '-2:word.islower()': word1.islower()
        })
    if i < len(sentence) - 1:
        word1 = sentence[i + 1][0]
        postag1 = sentence[i + 1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
            '+1:word.islower()': word1.islower()
        })
    else:
        features['EOS'] = True # EOS = End of Sentence
    if i < len(sentence) - 2:
        word1 = sentence[i + 2][0]
        postag1 = sentence[i + 2][1]
        features.update({
            '+2:word.lower()': word1.lower(),
            '+2:word.istitle()': word1.istitle(),
            '+2:word.isupper()': word1.isupper(),
            '+2:postag': postag1,
            '+2:postag[:2]': postag1[:2],
            '+2:word.islower()': word1.islower()
        })
    return features


def convert_data(data):
    sentences = []
    for d in data:
        sentences.append(list(zip(d[0], d[1], d[2])))
    x_data = [extract_sent_features(s) for s in sentences]
    y_data = [extract_labels(s) for s in sentences]
    return x_data, y_data

In [11]:
X_train, y_train = convert_data(train_data)

X_test, y_test = convert_data(test_data)

In [12]:
print(f'\nExample features: {X_train[0]}')

print(f'\nLabel: {y_train[0]}')


Example features: [{'bias': 1.0, 'word.lower()': 'ementa', 'word[-3:]': 'NTA', 'word[-2:]': 'TA', 'word.isupper()': True, 'word.istitle()': False, 'word.isdigit()': False, 'postag': 'PROPN', 'postag[:2]': 'PR', 'word.islower()': False, 'word[0].isupper()': True, 'word[0].islower()': False, 'not word[0].isalnum()': False, 'not word.isalnum()': False, 'word.isalpha()': True, 'BOS': True, '+1:word.lower()': ':', '+1:word.istitle()': False, '+1:word.isupper()': False, '+1:postag': 'PUNCT', '+1:postag[:2]': 'PU', '+1:word.islower()': False, '+2:word.lower()': 'apelação', '+2:word.istitle()': False, '+2:word.isupper()': True, '+2:postag': 'PROPN', '+2:postag[:2]': 'PR', '+2:word.islower()': False}, {'bias': 1.0, 'word.lower()': ':', 'word[-3:]': ':', 'word[-2:]': ':', 'word.isupper()': False, 'word.istitle()': False, 'word.isdigit()': False, 'postag': 'PUNCT', 'postag[:2]': 'PU', 'word.islower()': False, 'word[0].isupper()': False, 'word[0].islower()': False, 'not word[0].isalnum()': True, 

In [13]:
crf = CRF(max_iterations=100, c1=0.1, c2=0.1, all_possible_transitions=True)

# Testar com max_iterations=100 e all_possible_transitions=True

In [14]:
try:
  crf.fit(X_train, y_train)
except AttributeError:
  pass

In [15]:
y_pred = crf.predict(X_test)

In [16]:
report = classification_report(y_test, y_pred)

print(report)


                precision    recall  f1-score   support

JURISPRUDENCIA       0.75      0.71      0.73       185
    LEGISLACAO       0.84      0.80      0.82       378
         LOCAL       0.65      0.66      0.65        47
   ORGANIZACAO       0.89      0.78      0.83       501
        PESSOA       0.90      0.76      0.83       233
         TEMPO       0.92      0.78      0.84       192

     micro avg       0.86      0.77      0.81      1536
     macro avg       0.82      0.75      0.78      1536
  weighted avg       0.86      0.77      0.81      1536



In [17]:
import pandas as pd
from io import StringIO

report = classification_report(y_test, y_pred, output_dict=True)
results = []
for categoria, metrics in report.items():
    # Skip 'accuracy', 'macro avg', and 'weighted avg' keys
    if categoria in ('micro avg', 'macro avg', 'weighted avg'):
        continue

    precision = metrics['precision']
    recall = metrics['recall']
    f1_score = metrics['f1-score']
    support = metrics['support']
    results.append([categoria, precision, recall, f1_score, support])

df_results = pd.DataFrame(results, columns=['Categoria', 'Precisão', 'Recall', 'F1-Score', 'Support'])
arquivo_csv = f'PLN_TRABALHO-02-{corpus_name}.csv'
df_results.to_csv(arquivo_csv, index=False, encoding='utf-8')

print("CSV file "+arquivo_csv+" created successfully.")

CSV file PLN_TRABALHO-02-le_ner.csv created successfully.


In [18]:
def extract_ner(x, y, exclude=None, is_bio=False):
    if len(x) != len(y):
        print("\nERROR: 'x' and 'y' must be the same size.")
        return None
    if type(exclude) is not list:
        exclude = []  # Guarda os tokens que serão excluídos do resultado
    extracted_ners = []  # Guarda dicionários com as entidades nomeadas filtradas/agrupadas
    for i in range(len(y)):
        ind_last_token = len(y[i]) - 1
        j = 0
        same_entity = []  # Guarda os índices dos tokens vizinhos que pertencem à mesma entidade nomeada
        ners = {}  # Guarda o índice do token (como chave) e uma lista de tokens vizinhos que pertencem à mesma entidade
        has_began = False  # Indica se um label começou com 'B-', no caso do formato BIO
        # Percorre cada um dos tipos de tokens da sentença e, se houver, verifica se o proximo tipo de token é igual
        while j <= ind_last_token:
            if y[i][j] not in exclude:
                if j not in same_entity:
                    same_entity.append(j)
                    if is_bio and len(y[i][j]) >= 2 and y[i][j][:2].upper() == 'B-':  # Se é do formato BIO e é início
                        has_began = True
                if j + 1 <= ind_last_token:
                    if has_began and len(y[i][j + 1]) >= 2 and y[i][j + 1][:2].upper() == 'I-' \
                            and y[i][j][2:] == y[i][j + 1][2:]:
                        same_entity.append(j + 1)
                    elif not is_bio and y[i][j] == y[i][j + 1]:
                        same_entity.append(j + 1)
                    else:
                        ners[j] = same_entity
                        same_entity = []
                        has_began = False
                else:
                    ners[j] = same_entity
                    has_began = False
            j += 1
        # Processa as ners extraidas da sentença desta iteração
        if ners:
            sent = x[i]
            ners_aux = []
            for ind_token, filtered_tokens in ners.items():
                ner = ''
                for t in filtered_tokens:
                    ner += sent[t] + ' '
                # Trata o caso do formato BIO e retira os 'B-' e 'I-' dos nomes dos labels
                if is_bio and len(y[i][ind_token]) >= 2:
                    label = y[i][ind_token][2:]
                else:
                    label = y[i][ind_token]
                ners_aux.append((ner.strip(), label))
            extracted_ners.append(ners_aux)
        else:
            extracted_ners.append([])  # Para manter a ordem das sentenças, caso não seja extraida nenhuma ner
    return extracted_ners

In [19]:
ner_ext_test = extract_ner(test_data_original[:, 0],test_data_original[:, 1], exclude=['O'], is_bio=True)

ner_ext_pred = extract_ner(test_data_original[:, 0], y_pred, exclude=['O'], is_bio=True)

In [20]:
ner_ext_test[0]

[('ACÓRDÃO 1160/2016', 'JURISPRUDENCIA'),
 ('PLENÁRIO', 'ORGANIZACAO'),
 ('AUGUSTO NARDES', 'PESSOA'),
 ('Processo 006.010/2000-4', 'JURISPRUDENCIA'),
 ('11/05/2016', 'TEMPO'),
 ('Ana Arraes', 'PESSOA')]

In [21]:
ner_ext_pred[0]

[('ACÓRDÃO 1160/2016', 'JURISPRUDENCIA'),
 ('PLENÁRIO', 'ORGANIZACAO'),
 ('AUGUSTO NARDES', 'PESSOA'),
 ('Processo 006.010/2000-4', 'JURISPRUDENCIA'),
 ('11/05/2016', 'TEMPO'),
 ('Ana Arraes', 'PESSOA')]